# Test Pipeline SIPI-ETL con Scoring Religioso
Este notebook prueba el pipeline completo: scraping + scoring sin hardcode.

In [1]:
# 1. Librerías y path
import sys, tomllib, pandas as pd, pathlib
sys.path.insert(0, str(pathlib.Path.cwd().parent))

In [3]:
# 1.5 Verificar rutas reales
import os, pathlib
ruta_toml = pathlib.Path.cwd().parent / "src" / "modules" / "portals" / "idealista" / "config" / "scraping.toml"
print("Ruta absoluta:", ruta_toml.resolve())
print("¿Existe?:", ruta_toml.exists())
if ruta_toml.exists():
    print("Contenido:")
    print(ruta_toml.read_text())
else:
    print("NO EXISTE – vamos a crearlo")

Ruta absoluta: /home/jovyan/dev/sipi-etl/src/modules/portals/idealista/config/scraping.toml
¿Existe?: True
Contenido:
[scraping.idealista.test]
provincias = ["sevilla"]
tipos_inmueble = ["venta-edificios", "venta-terrenos"]
max_items_total = 10
max_pages_per_tipo = 2

[scraping.idealista.production]
provincias = ["sevilla", "cadiz", "malaga", "granada", "almeria", "jaen", "cordoba", "huelva"]
tipos_inmueble = ["venta-edificios", "venta-terrenos", "venta-locales", "venta-viviendas"]
max_items_total = 50000
max_pages_per_tipo = -1


In [5]:
# 2. Cargar configuración TOML (test) con ruta absoluta
%reload_ext autoreload
%autoreload 2
import tomllib, pathlib
ruta_toml = pathlib.Path("/home/jovyan/dev/sipi-etl/src/modules/portals/idealista/config/scraping.toml")
with ruta_toml.open("rb") as f:
    config = tomllib.load(f)["scraping"]["idealista"]["test"]

In [15]:
# 3. Importar scraper (sin puntitos, relativo al sys.path)
from src.modules.portals.idealista.extract.scraper import IdealistaScraper
from src.modules.portals.idealista.extract.idealista_client import IdealistaClient

ModuleNotFoundError: No module named 'transform'

In [ ]:
# 4. Ejecutar scraping + scoring
client = IdealistaClient(headless=True, use_selenium=True)
scraper = IdealistaScraper()
scraper.client = client
resultados = await scraper.scrape(**config)

In [ ]:
# 5. Ver resultado
df = pd.DataFrame(resultados)
print("Total inmuebles:", len(df))
display(df.head())
display(df[['id_idealista','titulo_completo','score_religioso','evidencias_religiosas','latitud','longitud','precision_geo']].dropna())

In [ ]:
# 6. Guardar con scoring
df.to_csv("scraping_con_score.csv", index=False, sep=";", encoding="utf-16")